In [ ]:
import workingmem
from workingmem.utils import get_wandb_runs
from pathlib import Path
import pandas as pd
import yaml

In [ ]:
# project_name = "wm-comp-limit-7.4.1c1_pt"
project_name = "wm-comp-limit-7.4.1c0"
# project_name = "wm-comp-limit-7.3.2"
# stores td_prob, role_n_congruence[ , n_back ]
# sweep_data = [
#     {"sweep_id": "be131vw7", "td_prob": 0, "role_n_congruence": 0},
#     {"sweep_id": "m8c2cr7g", "td_prob": 0, "role_n_congruence": 0.25},
#     {"sweep_id": "lezq9cbe", "td_prob": 0, "role_n_congruence": 0.5},
#     {"sweep_id": "8g76x2z1", "td_prob": 0, "role_n_congruence": 0.75},
#     {"sweep_id": "pedc15j6", "td_prob": 0, "role_n_congruence": 1},
#     {"sweep_id": "crjufe02", "td_prob": 1, "role_n_congruence": 0},
#     {"sweep_id": "lpgcp60v", "td_prob": 1, "role_n_congruence": 0.25},
#     {"sweep_id": "hzflaqxx", "td_prob": 1, "role_n_congruence": 0.5},
#     {"sweep_id": "ck9pi0go", "td_prob": 1, "role_n_congruence": 0.75},
#     {"sweep_id": "w03izn1c", "td_prob": 1, "role_n_congruence": 1},
# ]

with open(
    (
        config_fname
        # := "../../../../configs/ref_n_back_finetuning_rnn_4to5.yaml_created_configs/26-02-04-12-27_sweep_dict.yaml"
        := "../../../../configs/ignore_aware_ref_n_back_rnn_4_role_pretraining.yaml_26-02-03_sweep_dict.yaml"
    ),
    # "../../../../configs/ref_n_back_finetuning_4to5.yaml_25-12-03_sweep_dict.yaml"
    # "../../../../configs/ignore_aware_ref_n_back_hparams.yaml_25-11-26_sweep_dict.yaml",
    # "../../../../configs/ref_n_back.yaml_25-11-19_sweep_dict.yaml",
    "+r",
) as f:
    sweep_data = yaml.load(f, yaml.SafeLoader)

sweep_data

In [ ]:
# prefix = Path(Path(config_fname).parent.stem).stem
prefix = Path(Path(config_fname).stem).stem
Path(prefix).mkdir(exist_ok=True)

for record in sweep_data:
    sweep_id, td_prob, role_n_congruence, n_back, concurrent_reg = (
        record["sweep_id"],
        record["dataset.td_prob"],
        record["dataset.role_n_congruence"],
        record["dataset.n_back"],
        record.get("dataset.concurrent_reg", None),
    )
    print(f"fetching for: {sweep_id=}, {td_prob=}, {role_n_congruence=}")

    try:
        df = get_wandb_runs(project_name, sweep_id)
        df["td_prob"], df["role_n_congruence"] = (
            td_prob,
            role_n_congruence,
        )
        df["from_pretrained"] = record.get("model.from_pretrained", None)

        fname = f"{prefix}/{sweep_id}_tdprob={td_prob}_rolencong={role_n_congruence}_nback={n_back}_roles=50_concurrent={concurrent_reg}.csv"
        print(f"saving: {fname}")
        df.to_csv(fname)
    except KeyError as e:
        print(e, sweep_id, sweep_data[sweep_id])